# Robot Motion Planning

## Function Specifications

Goal is to find a shortest path through a maze/grid in which there are hurdles as well. Grid/Maze format:
- 0 = Navigable space
- 1 = Occupied space

Define a function, search() that returns a list in the form of [optimal path length, row, col]. For the grid shown below, function should output [11, 4, 5].

If there is no valid path from the start point to the goal, function should return the string 'fail'.

![visualization of maze](visualization.png)

In [1]:
#function to calculate next cell after specificed move has been made
def get_next_cell(current_cell, move):
    next_cell = [current_cell[0] + move[0], current_cell[1] + move[1]]
    return next_cell

In [2]:
#Function to check if making the passed move is possible and valid
def is_move_valid(grid, current_cell, move):
    #calculate the next cell
    next_cell = get_next_cell(current_cell, move)
    
    #make sure we are not overshooting the maze/grid boundary
    if next_cell[0] >= len(grid) or next_cell[1] >= len(grid[0]):
        return False
    
    #make sure we are not undershooting the maze/grid boundary
    if next_cell[0] < 0 or next_cell[1] < 0:
        return False
    
    #make sure the cell is not already occupied, either because 
    #it is an obstacle or we traveresed it before
    if grid[next_cell[0]][next_cell[1]] == 1:
        return False
    
    return True

In [3]:
#function to make a move, it assumes that the move
#passed is a valid move to make
def make_move(grid, current_cell, move):
    #calculate the next cell
    #calculate the next cell
    next_cell = get_next_cell(current_cell, move)
    
    #update the grid cell as marked
    grid[next_cell[0]][next_cell[1]] = 1

In [4]:
#function to find possible adjacent cells which are not occupied
def find_available_cells(grid, current_cell, moves):
    available_cells = []
    
    for move in moves:
        if (is_move_valid(grid, current_cell, move)):
            available_cells.append(get_next_cell(current_cell, move))
            
    return available_cells

In [5]:
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, function should return the string
# 'fail'
# ----------

# Grid/Maze format:
#   0 = Navigable space
#   1 = Occupied space

grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 0]]

# grid = [[0, 1],
#         [0, 0]]

# grid = [[0, 1, 1, 1, 1],
#         [0, 1, 0, 0, 0],
#         [0, 0, 0, 1, 0],
#         [1, 1, 1, 1, 0],
#         [0, 0, 0, 1, 0]]

init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

In [6]:
def add_cells_to_open_list(open_cells, cells, new_cost):
    for cell in cells:
        open_cells.append([new_cost, cell[0], cell[1]])

In [7]:
def find_min_cost_cell(open_cells):
    min_cell = open_cells[0]
    for cell in open_cells:
        #index 0 of each cell is actually cost
        if(cell[0] < min_cell[0]):
            min_cell = cell
            
    return min_cell

In [8]:
def search(grid,init,goal,cost, delta):
    #list to store the path of cells traversed 
    path = []
    #list of open cells that can be traveresed
    open_cells = []
    
    #create a list to keep track of nodes
    expand = [[-1 for c in range(len(grid[0]))] for r in range(len(grid))]
    
    #variable to keep track of cells visited count
    visited_cells_count = 0

    #current cell to be visited
    current_cell = init
    
    #last visited cell
    last_cell = init
    
    #mark current cell
    grid[current_cell[0]][current_cell[1]] = 1
    
    #place the cell count number to current visited cell
    expand[current_cell[0]][current_cell[1]] = visited_cells_count
    
    #increase cells counter by 1
    visited_cells_count += 1
    
    #add current cell to path as it is traversed 
    path.append([cost, current_cell[0], current_cell[1]])
    
    #find next possible candidate cells that are not occupied and adjacent to current cell
    next_cells = find_available_cells(grid, current_cell, delta)
    #mark available cells as visited
    for cell in next_cells:
        r = cell[0]
        c = cell[1]
        grid[r][c] = 1
        
    #add candidate cells to list of open cells
    add_cells_to_open_list(open_cells, next_cells, cost)
    
    while(len(open_cells) > 0):
        #find a minimum cost cells from the list of cells
        current_cell = find_min_cost_cell(open_cells)
    
        open_cells.remove(current_cell)
        
        #mark current cell
        grid[current_cell[1]][current_cell[2]] = 1
        
        #place the cell count number to current visited cell
        expand[current_cell[1]][current_cell[2]] = visited_cells_count
        #increase cells counter by 1
        visited_cells_count += 1
        
        #add current [cell.cost+1, cell] to path as it is traversed 
        path.append([current_cell[0], current_cell[1], current_cell[2]])
        
        #find next possible candidate cells that are not occupied and adjacent to current cell
        next_cells = find_available_cells(grid, [current_cell[1], current_cell[2]], delta)
        
        #mark available cells as visited
        for cell in next_cells:
            r = cell[0]
            c = cell[1]
            grid[r][c] = 1
        
        #add candidate cells to list of open cells
        add_cells_to_open_list(open_cells, next_cells, current_cell[0] + 1)
        
        last_cell = current_cell
        
        if [last_cell[1], last_cell[2]] == goal:
            return path, expand
    
    return None, expand

In [9]:
import pprint

path, expand = search(grid, init, goal, cost, delta)
if path == None:
    print ('fail')
else:
    print(path[len(path)-1])
    
print(path)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(expand)

# is_move_valid(grid, init, delta[3])
# make_move(grid, init, delta[3])
# find_available_cells(grid, init, delta)

# cells1 = search(grid, init, goal, 4, delta)
# cells = cells + cells1
# cell = find_min_cost_cell(cells)
# print(cell)
# cells.remove(cell)
# print(cells)
# print(cells[0])
# cells.remove(cells[0][0])
# print(cells)
# print(grid)
# print(delta[0])
# print(grid[0][1])


[11, 4, 5]
[[1, 0, 0], [1, 1, 0], [1, 0, 1], [2, 2, 0], [2, 1, 1], [3, 3, 0], [3, 2, 1], [4, 4, 0], [4, 3, 1], [4, 2, 2], [5, 4, 1], [5, 2, 3], [6, 4, 2], [6, 1, 3], [7, 4, 3], [7, 0, 3], [7, 1, 4], [8, 0, 4], [8, 1, 5], [9, 0, 5], [9, 2, 5], [10, 3, 5], [11, 4, 5]]
[   [0, 2, -1, 15, 17, 19],
    [1, 4, -1, 13, 16, 18],
    [3, 6, 9, 11, -1, 20],
    [5, 8, -1, -1, -1, 21],
    [7, 10, 12, 14, -1, 22]]


In [10]:
a = [[0, 1],[1, 0]]

for r in range(len(a)):
    for c in range(len(a[0])):
        a[r][c] = -1
        
a = [[-1 for row in range(len(grid[0]))] for c in range(len(grid))]
pp.pprint(a)

[   [-1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1]]


## Using Heuristic Function to Optimize Search

The heuristic function is a way to inform the search about the direction to a goal. It provides an informed way to guess which neighbor of a node will lead to a goal. There is nothing magical about a heuristic function.